In [1]:
#!/usr/bin/env python
from __future__ import print_function

import numpy as np
import pandas as pd
import h5py
import re

/home/rhf/.local/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from datetime import datetime

f = h5py.File("/SNS/MANDI/IPTS-8776/0/5800/NeXus/MANDI_5800_event.nxs", "r")


event_banks = [item for item in f["entry"].values() if isinstance(item, h5py.Group) and item.name.endswith("events")]

df = pd.DataFrame()


In [3]:
for bank in event_banks[-1:]: # For debug use event_banks[-1:]
    
    #
    # index is event id, value is pixel_id
    event_pixel_id = bank["event_id"].value
    
    #
    # create tmp df with the pixel_id inside
    df_tmp = pd.DataFrame(
        data = event_pixel_id,
        columns = ['pixel_id']
    )
    
    #
    # Get the Pixels position 256x256 = 65536
    pixel_ids = bank["pixel_id"].value
    # flatten pixel_ids and get respective indices
    XX,YY = np.meshgrid(np.arange(pixel_ids.shape[1]),np.arange(pixel_ids.shape[0]))
    # 2d array of the format: pixel_id, i, j
    pixel_id_and_indices = np.vstack((pixel_ids.ravel(),XX.ravel(),YY.ravel())).T
    
    # tmp df for detector info
    df_tmp2 = pd.DataFrame(
        data=pixel_id_and_indices,
        columns=['pixel_id', 'i', 'j'],
    )
    # Left join
    df_tmp = pd.merge(left=df_tmp,right=df_tmp2, how='left', left_on='pixel_id', right_on='pixel_id')
    # remove the pixel_id (we don't need it amy more)
    df_tmp = df_tmp.drop('pixel_id', 1)

    #
    # index is event id, value time
    event_time_offset = bank["event_time_offset"].value
    df_tmp['time_offset'] = event_time_offset





    # Length pulses != events
    #
    # index is pulse id, value is event_id
    pulse_event_index = bank["event_index"].value
    pulse_event_index_step = np.diff(pulse_event_index) # difference between the current and previous
    pulse_event_index_step = np.insert(pulse_event_index_step,0,0) # Insert 0 in the beggining
    #
    # index is pulse id, value pulse 0 time
    pulse_time_zero = bank["event_time_zero"].value
    pulse_time_zero_with_event = pulse_time_zero[pulse_event_index_step>0] # only where there is a count
    pulse_event_index_with_event = pulse_event_index[pulse_event_index_step>0]

    df_tmp2 = pd.DataFrame(
        data= np.column_stack((pulse_event_index_with_event,pulse_time_zero_with_event)),
        columns=['event_id', 'time_zero'],
    )
    df_tmp2.event_id = df_tmp2.event_id.astype(np.int64)
    # Left join
    df_tmp = pd.merge(left=df_tmp,right=df_tmp2, how='left',  left_index = True, right_on='event_id')
    df_tmp = df_tmp.fillna(method='bfill') # fill nans with the value from the next cell



    #
    # get the bank number
    pattern = re.compile(r"/entry/bank(\d+)_events")
    match = pattern.match(bank.name)
    bank_number = int(match.group(1))

    #
    # Bank Number
    bank_number_array = np.array([bank_number]*len(df_tmp))
    df_tmp['bank_id'] = bank_number_array

    # Append to the final table
    df = df.append(df_tmp, ignore_index = True)

In [4]:
df

,i,j,time_offset,event_id,time_zero,bank_id
0,215,76,13546.000000,0,0.333328,8
1,27,70,12467.299805,1,0.333328,8
2,77,77,11832.000000,2,0.349994,8
3,96,109,14687.799805,3,0.366660,8
4,227,37,13229.400391,4,0.416659,8
5,169,146,18990.599609,5,0.433326,8
6,168,198,21495.000000,6,0.433326,8
7,218,119,11197.599609,7,0.433326,8
8,246,167,14558.599609,8,0.449992,8
9,37,186,12407.400391,9,0.449992,8


In [5]:
#
# Convert TOF number to time_delta
# SLOW!
df['time_offset'] = pd.to_timedelta(df['time_offset'],  unit="us")

# SLOW
start_date = np.datetime64(f['entry']['start_time'].value[0])
df['time_zero'] = pd.to_timedelta(df.time_zero) + start_date
df = df.set_index('time_zero')

# We don't need event id
df = df.drop('event_id', 1)
df

/home/rhf/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  import sys


,i,j,time_offset,bank_id
time_zero,,,,
2016-07-15 06:50:28.825350666,215,76,00:00:00.013546,8
2016-07-15 06:50:28.825350666,27,70,00:00:00.012467,8
2016-07-15 06:50:28.825350666,77,77,00:00:00.011832,8
2016-07-15 06:50:28.825350666,96,109,00:00:00.014687,8
2016-07-15 06:50:28.825350666,227,37,00:00:00.013229,8
2016-07-15 06:50:28.825350666,169,146,00:00:00.018990,8
2016-07-15 06:50:28.825350666,168,198,00:00:00.021495,8
2016-07-15 06:50:28.825350666,218,119,00:00:00.011197,8
2016-07-15 06:50:28.825350666,246,167,00:00:00.014558,8
